In [1]:
import os
import csv
import math
import random
import operator
import timeit
import numpy as np


In [2]:
#root = 'C:/Users/sezin/Desktop/SON SENE/CS464-Machine Learning/HW1/q2/'
#csv_path_training_data = os.path.join(root, 'diabetes_train_features.csv')
#csv_path_test_data = os.path.join(root, 'diabetes_test_features.csv')
#csv_path_training_label = os.path.join(root, 'diabetes_train_labels.csv')
#csv_path_test_label = os.path.join(root, 'diabetes_test_labels.csv')
csv_path_training_data = 'diabetes_train_features.csv'
csv_path_test_data = 'diabetes_test_features.csv'
csv_path_training_label =  'diabetes_train_labels.csv'
csv_path_test_label =  'diabetes_test_labels.csv'
#with open(csv_path_training_data, 'r') as csvfile:
#    lines = csv.reader(csvfile)
#    for row in lines:
#        print(row)

In [3]:
def get_dataset(filename):
     with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        next(lines)
        dataset = list(lines)
        data = np.array([[float(row[col_i]) for col_i in range(1,len(row))] for row in dataset])
        #print(data)
        return data

In [4]:
train_features = get_dataset(csv_path_training_data)
test_features = get_dataset(csv_path_test_data)
train_labels = get_dataset(csv_path_training_label)
test_labels = get_dataset(csv_path_test_label)

#print(test_features[1])

In [5]:
def euclidean_distance(sample1, sample2):
    #print(len(sample1))
    #print(len(sample2))
    assert len(sample1) == len(sample2), "Different number of features exist for the given samples"
    distance = 0
    for i in range(0,len(sample1)): #first num -> indicates the num of the sample 
        distance += pow((sample1[i] - sample2[i]), 2)
    return math.sqrt(distance)

In [6]:
train_features.shape

(614, 8)

In [7]:
def get_neighbors(train_features, train_labels, test_sample, k):
    distances = []
    neighbors = []
    
    for train_sample, sample_label,  in zip(train_features, train_labels):
        #print(train_sample)
        #print(test_sample)
        distance = euclidean_distance(train_sample, test_sample)
        distances.append((train_sample, sample_label, distance))
        
    distances.sort(key=operator.itemgetter(2))
    
    for i in range(k):
        neighbors.append(distances[i])

    #print(distances[:k])
    return distances[:k]

In [8]:
test1_neighbors = get_neighbors(train_features,train_labels,test_features[1],9)

In [9]:
classes = np.unique(train_labels)
print(classes)

[0. 1.]


In [10]:
def classify(neighbors, classes):
    class_votes = {cls: 0 for cls in classes}
    
    for n in neighbors:
        class_votes[int(n[1])] += 1
    
    most_vote_count = 0
    most_voted_class = None
    for cls in class_votes:
        if class_votes[cls] > most_vote_count:
            most_voted_class = cls
            most_vote_count = class_votes[cls]
    return most_voted_class
    

In [11]:
classify(test1_neighbors, np.unique(train_labels))


0.0

In [12]:
def calc_accuracy(gt_y, pred_y):
    correct = 0
    for g_y, p_y in zip(gt_y, pred_y):
        if g_y == p_y: 
            correct += 1
            
    return (correct/float(len(gt_y))*100)

def confusion_matrix(correct_labels, predictions):
    confusion = [[0,0], [0, 0]]
    #tp fn; fp tn
    for l, p in zip(correct_labels, predictions):
        if l == p:
            if l == 0: #true negative
                confusion[1][1] += 1
            else:      #true positive
                confusion[0][0] += 1
        else: 
            if l == 0: #false positive
                confusion[1][0] += 1
            else: # false negative
                confusion[0][1] += 1
                
    return confusion

In [13]:
class KNN:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        
    def fit(self, X, y):
        start = timeit.default_timer()
        self.train_X = X
        self.train_y = y
        self.classes = np.unique(y)
        stop = timeit.default_timer()
        print('training time: ', stop - start)
        
    # X: test data
    def predict(self, X):
        start = timeit.default_timer()
        predictions = []
        for x in X:
            neighbors = get_neighbors(self.train_X, self.train_y, x, self.n_neighbors)
            p_class = classify(neighbors, self.classes)
            predictions.append(p_class)
        stop = timeit.default_timer()
        print('validation time: ', stop - start)
        return predictions     

In [14]:

knn = KNN(n_neighbors=9)
knn.fit(train_features, train_labels)
pred_y = knn.predict(test_features)
acc = calc_accuracy(test_labels, pred_y)
print(acc)
print(confusion_matrix(test_labels,pred_y))


training time:  0.0002649999805726111
validation time:  1.2038394999981392
72.07792207792207
[[37, 18], [25, 74]]


In [15]:
def eliminate_feature(feature, x):
    if x == 0:
        return np.array([[float(row[col_i]) for col_i in range(1,len(row))] for row in feature])
    elif x == feature.shape[1]-1:
        return np.array([[float(row[col_i]) for col_i in range(0,len(row)-1)] for row in feature])
    elif x < feature.shape[1]-1:
        firstPart = np.array([[float(row[col_i]) for col_i in range(0,x)] for row in feature])
        secondPart = np.array([[float(row[col_i]) for col_i in range(x+1,len(row))] for row in feature])
        return np.concatenate((firstPart, secondPart), axis=1)
    else:
        return feature
    
def backward_elimination(training_feature, test_feature, training_label, test_label):

    knn = KNN(n_neighbors=9)
    knn.fit(training_feature, training_label)
    pred_y = knn.predict(test_feature)
    accuracy_before = calc_accuracy(test_label,pred_y)
    accuracy = accuracy_before
    print('Accuracy before elimination ' + str(accuracy))

    max_accuracy_this_loop = accuracy;
    eliminateThis = 0;
    
        #Eliminate 1 column & try again
        
    for k in range(0,len(training_feature[0])):      
        eliminated_test = eliminate_feature(test_feature,k)
        eliminated_training = eliminate_feature(training_feature,k)
        new_knn = KNN(n_neighbors = 9)
        new_knn.fit(eliminated_training,training_label)
        new_prediction = new_knn.predict(eliminated_test)
        accuracy = calc_accuracy(test_label, new_prediction)
        print('k ' + str(k) +' accuracy this loop '+ str(accuracy)+ ' max accuracy this loop '+ str(max_accuracy_this_loop)+ ' eliminateThis ' + str(eliminateThis))
        print('confusion matrix this loop')
        print(confusion_matrix(test_labels,new_prediction))
        if (accuracy > max_accuracy_this_loop):
            max_accuracy_this_loop = accuracy
            eliminateThis = k
                
    if(max_accuracy_this_loop > accuracy_before):
        
        print('eliminating: ' + str(eliminateThis) )
        print()
        return eliminate_feature(training_feature, eliminateThis), eliminate_feature(test_feature, eliminateThis), training_label, test_label, max_accuracy_this_loop
    else :
        return training_feature, test_feature, training_label, test_label, accuracy_before
    

In [16]:
#without anything eliminated
knn1 = KNN(n_neighbors = 9)
knn.fit(train_features,train_labels)
pred = knn.predict(test_features)
max_acc = calc_accuracy(test_labels, pred)
accuracy_RN = max_acc
print(max_acc)
print()

eliminated_training = train_features
eliminated_test = test_features

while True:
    if(max_acc <= accuracy_RN):
        a = backward_elimination(eliminated_training,eliminated_test, train_labels, test_labels)
        accuracy_RN = a[4]
        eliminated_training = a[0]
        eliminated_test = a[1]
        if(accuracy_RN > max_acc):
            max_acc = accuracy_RN
        else:
            break;
    else:
        break;
        
print('bitti')
    

training time:  0.000371300004189834
validation time:  1.0916233000170905
72.07792207792207

training time:  0.0009030000073835254
validation time:  0.9561219000024721
Accuracy before elimination 72.07792207792207
training time:  8.290002006106079e-05
validation time:  0.8352437999856193
k 0 accuracy this loop 71.42857142857143 max accuracy this loop 72.07792207792207 eliminateThis 0
confusion matrix this loop
[[36, 19], [25, 74]]
training time:  0.0003583999932743609
validation time:  1.4655023000086658
k 1 accuracy this loop 60.3896103896104 max accuracy this loop 72.07792207792207 eliminateThis 0
confusion matrix this loop
[[20, 35], [26, 73]]
training time:  0.0006762999983038753
validation time:  1.085496499988949
k 2 accuracy this loop 68.83116883116884 max accuracy this loop 72.07792207792207 eliminateThis 0
confusion matrix this loop
[[34, 21], [27, 72]]
training time:  0.00012070001685060561
validation time:  1.0026731000107247
k 3 accuracy this loop 70.12987012987013 max accu